# Module 1 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
.       plains     1
*       forest     3
#       hills      5
~       swamp      7
x       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
....*..
...***.
.###...
..##...
..#..**
....***
.......
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [1]:
full_world = [
  ['.', '.', '.', '.', '.', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', '.', '.', '.', '*', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '.', '.'], 
  ['.', '.', '.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '#', '#', '#', 'x', 'x', '#', '#'], 
  ['.', '.', '.', '.', '#', 'x', 'x', 'x', '*', '*', '*', '*', '~', '~', '*', '*', '*', '*', '*', '.', '.', '#', '#', 'x', 'x', '#', '.'], 
  ['.', '.', '.', '#', '#', 'x', 'x', '*', '*', '.', '.', '~', '~', '~', '~', '*', '*', '*', '.', '.', '.', '#', 'x', 'x', 'x', '#', '.'], 
  ['.', '#', '#', '#', 'x', 'x', '#', '#', '.', '.', '.', '.', '~', '~', '~', '~', '~', '.', '.', '.', '.', '.', '#', 'x', '#', '.', '.'], 
  ['.', '#', '#', 'x', 'x', '#', '#', '.', '.', '.', '.', '#', 'x', 'x', 'x', '~', '~', '~', '.', '.', '.', '.', '.', '#', '.', '.', '.'], 
  ['.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '.', '.', '#', 'x', 'x', 'x', '~', '~', '~', '.', '.', '#', '#', '#', '.', '.'], 
  ['.', '.', '.', '#', '#', '#', '.', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', '.', '~', '~', '.', '.', '#', '#', '#', '.', '.', '.'], 
  ['.', '.', '.', '~', '~', '~', '.', '.', '#', '#', '#', 'x', 'x', 'x', 'x', '.', '.', '.', '~', '.', '#', '#', '#', '.', '.', '.', '.'], 
  ['.', '.', '~', '~', '~', '~', '~', '.', '#', '#', 'x', 'x', 'x', '#', '.', '.', '.', '.', '.', '#', 'x', 'x', 'x', '#', '.', '.', '.'], 
  ['.', '~', '~', '~', '~', '~', '.', '.', '#', 'x', 'x', '#', '.', '.', '.', '.', '~', '~', '.', '.', '#', 'x', 'x', '#', '.', '.', '.'], 
  ['~', '~', '~', '~', '~', '.', '.', '#', '#', 'x', 'x', '#', '.', '~', '~', '~', '~', '.', '.', '.', '#', 'x', '#', '.', '.', '.', '.'], 
  ['.', '~', '~', '~', '~', '.', '.', '#', '*', '*', '#', '.', '.', '.', '.', '~', '~', '~', '~', '.', '.', '#', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', 'x', '.', '.', '*', '*', '*', '*', '#', '#', '#', '#', '.', '~', '~', '~', '.', '.', '#', 'x', '#', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '#', '#', '.', '~', '.', '#', 'x', 'x', '#', '.', '.', '.'], 
  ['.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '.', '.', 'x', 'x', 'x', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', 'x', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', '.', '#', '#', '.', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '~', '~', '~', '~'], 
  ['.', '.', '#', '#', '#', '#', 'x', 'x', '*', '*', '*', '*', '*', '.', 'x', '.', '.', '.', '.', '.', '~', '~', '~', '~', '~', '~', '~'], 
  ['.', '.', '.', '.', '#', '#', '#', 'x', 'x', 'x', '*', '*', 'x', 'x', '.', '.', '.', '.', '.', '.', '~', '~', '~', '~', '~', '~', '~'], 
  ['.', '.', '.', '.', '.', '.', '#', '#', '#', 'x', 'x', 'x', 'x', '.', '.', '.', '.', '#', '#', '.', '.', '~', '~', '~', '~', '~', '~'], 
  ['.', '#', '#', '.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', '#', '#', '.', '~', '~', '~', '~', '~'], 
  ['#', 'x', '#', '#', '#', '#', '.', '.', '.', '.', '.', 'x', 'x', 'x', '#', '#', 'x', 'x', '.', 'x', 'x', '#', '#', '~', '~', '~', '~'], 
  ['#', 'x', 'x', 'x', '#', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', 'x', 'x', '#', '#', '#', '#', 'x', 'x', 'x', '~', '~', '~', '~'], 
  ['#', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '#', '#', '#', '.', '.', '.']]

## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [2]:
test_world = [
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '.', '.', '.', '.', '.', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
]

**what do you expect the policy would be?** Think about it for a bit. This will help you with your programming and debugging.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [3]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]  ## order is left, up, right, below

## Costs

We can encode the costs described above in a `Dict`:

In [4]:
COSTS = { '.': 1, '*': 3, '#': 5, '~': 7}

## A\* Search Implementation

As Python is an interpreted language, you're going to need to insert all of your helper functions *before* the actual `a_star_search` function implementation.

Please **read the Blackboard** for information about the expected code structure. I expect a "literate" style of "functional" programming (nothing fancy: only define functions, no class definitions, pass state between functions, isolate side effects like printing). Notebooks do not lend themselves to OOP very well.

-----

In [1]:
# add as many markdown and code cells here as you need for helper functions.

**successor**

The `successor` function returns the a list of all valid moves that can be made from the current node. 
It determines if the successor node is valid by checking if it is in the explored/frontier list or if it is an impassible terrain.

* **world** is the starting state representation for a navigation problem.
* **current_node** is the node for which we want to determine the successors.
* **moves** is the legal movement model expressed in offsets.
* **explored** is the list of all explored nodes
* **frontier** is the list nodes that needs to be explored
* **parent_node** is the dictionary of all coordinate's parent node

For example, for the test world if the current node is the node on the top left corner:

```
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '.', '.', '.', '.', '.', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],

```

it would return:

`[(1,0), (0,1)]` and the updateed `parent_node` dictionary

In [6]:
def successor(world, current_node, moves, frontier, explored, parent_node):
    valid_moves = []
    
    for move in moves:
        frontier_check = True   # False = node already in the frontier list
        explored_check = True   # False = node already in the explored list
        
        tmp_move =  (current_node[1][0]+move[0], current_node[1][1]+move[1])
        
        # check if node is in the explored list
        for item in explored:
            if tmp_move  == item[1]:
                if item[0] < current_node[0]:
                    parent_node[current_node[1]] = item[1]
                explored_check = False
                break
    
        # chek if node is in the frontier list
        for item in frontier:
            if tmp_move  == item[1]:
                frontier_check = False
                break

        # check if passes the frontier/explored check and make sure the move is in the known environment
        if tmp_move[0] >= 0 and tmp_move[1] >= 0 and tmp_move[0] < len(world) and tmp_move[1] < len(world) and world[tmp_move[0]][tmp_move[1]] != 'x' and explored_check == frontier_check == True:
            valid_moves += [tmp_move]
            
    return valid_moves, parent_node

**my_heuristic**

The `my_heurisic` an estimate of the total cost $f(x) = g(x)  + h(x)$ from the start to the goal through the current node, $x$. It calculates $h(x)$ by calculating euclidean distance from the succ_node to the goal. $g(x)$ is calculated by taking the distance from start to current_node + terrain_cost to successor.

Paramertes:
* **world** is the starting state representation for a navigation problem.
* **current_node** is the node for which we want to determine the successors.
* **succ_node** is the $x$ for which we wan to calculate $f(x)$
* **goal** is the desired end position `(x,y)`
* **costs** is a `Dict` of costs for each type of terrain.

retuns
it returns $h(x)$ and $g(x)$ for the successor node($x$)

In [7]:
# change the formal arguments and the return value to be what you need.
def my_heuristic(world, current_node, succ_node, goal, costs):
    terrain_type = world[succ_node[0]][succ_node[1]]
    hn = ((succ_node[0]-goal[1])**2 + (succ_node[1]-goal[0])**2)**0.5 # euclidean distance from succ node to goal
    gn = costs[terrain_type] + current_node[2]     # calcualte g(n)
    return  hn, gn

**a_star_search**

The `a_star_search` function uses the A\* Search algorithm to solve a navigational problem for an agent in a grid world. It calculates a path from the start state to the goal state and returns the actions required to get from the start to the goal. It is complete and it gives the optimal path from start to goal. 

* **world** is the starting state representation for a navigation problem.
* **start** is the starting location, `(x, y)`.
* **goal** is the desired end position, `(x, y)`.
* **costs** is a `Dict` of costs for each type of terrain.
* **moves** is the legal movement model expressed in offsets.
* **heuristic** is a heuristic function that returns an estimate of the total cost $f(x)$ from the start to the goal through the current node, $x$. The heuristic function might change with the movement model.


The function returns the offsets needed to get from start state to the goal as a `List`. For example, for the test world:

```
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '.', '.', '.', '.', '.', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],

```

it would return:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

Do not make unwarranted assumptions. For example, do not assume the starting point is always `(0, 0)` or that the goal is always in the lower right hand corner.

In [8]:
def a_star_search( world, start, goal, costs, moves, heuristic):
    parent_node = {}  # keeps tracks of parent node of each node
    parent_node[(start[1],start[0])] = None
    frontier = [(0, (start[1],start[0]), 0)]
    explored = []
    
    while frontier != []:
        current_node = frontier.pop(0)
        if current_node[1] == (goal[1], goal[0]):
            break
        valid_moves, parent_node = successor(world, current_node, moves, frontier, explored, parent_node)
        for move in valid_moves:
            hn, gn = heuristic(world, current_node, move, goal, costs)
            frontier += [(hn+gn, move, gn)]
            parent_node[move] = current_node[1]    # update the parent node
        explored += [current_node]
        frontier.sort(key = lambda x: x[0])    # inplace sort 
    
    # backtrace the path from goal to start
    optimal_path = []
    optimal_path = [(goal[1],goal[0])] 
    node = (goal[1],goal[0])    
    while node!= (start[1],start[0]):
        node = parent_node[node]
        optimal_path.insert(0,node)  # insert at the beginning of the list
    
    # calculate the actions from the known path
    actions = []
    for indx in range(len(optimal_path)-1):
        current_node, next_node =  optimal_path[indx], optimal_path[indx+1]
        val0, val1 = next_node[0]-current_node[0], next_node[1]-current_node[1]
        actions +=[(val0, val1)]
        
    return actions

**pretty_print_solution**

The `pretty_print_solution` function prints an ASCII representation of the solution generated by the `a_star_search`. For example, for the test world, it would take the `world` and `path` and print:

```
v******
v******
v******
>>>>>>v
******v
******v
******G
```

using `v`, `^`, `>`, `<` to represent actions and `G` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.


Note that in Python:
```
> a = ["*", "-", "*"]
> "".join(a)
*-*
```
Do not print raw data structures; do not insert unneeded/requested spaces!

In [9]:
def pretty_print_solution( world, path, start):
    directions_dict = {(0,1):">", (0,-1):"<", (1,0):"v", (-1,0):"^"}
    show_path = list(world)
    
    c_node = (start[1], start[0])
    for direction in path:
        show_path[c_node[0]][c_node[1]] = directions_dict[direction]
        c_node = (c_node[0] + direction[0], c_node[1]+direction[1])
    show_path[c_node[0]][c_node[1]] = 'G'
    
    for row in show_path:
        print("".join(row))

Execute `a_star_search` and `print_path` for the `test_world` and the `real_world`.

In [10]:
path = a_star_search(test_world, (0, 0), (len(test_world[0]) - 1, len(test_world) - 1), COSTS, MOVES, my_heuristic)
print(path)
pretty_print_solution(test_world, path, (0, 0))

[(1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0)]
v******
v******
v******
>>>>>>v
******v
******v
******G


In [11]:
path = a_star_search(full_world, (0, 0), (len(full_world[0]) - 1, len(full_world) - 1), COSTS, MOVES, my_heuristic)
print(path)
pretty_print_solution(full_world, path, (0, 0))

[(1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1), (0, 1), (-1, 0), (0, 1), (0, 1), (0, 1), (0, 1), (-1, 0), (0, 1), (0, 1), (-1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (0, 1), (1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1)]
v....**********............
v......*********..xxxxxxx..
v...xx***********xxx###xx##
v...#xxx****~~*****..##xx#.
v..##xx**..~~~~***...#xxx#.
v###xx##....~~~~~.....#x#..
v##xx##....#xxx~~~.....#...
v.#####......#xxx~~~..###..
v..###......##xx.~~..###...
v..~~~..###xxxx...~.###....
v.~~~~~.##xxx#.....#xxx#...
v~~~~~..#xx#....~~..#xx#...
v~~~~..##xx#.~~~~...#x#....
v~~~~..#**#....~~~~..#.....
>v..x..****####.~~~..#x#...
.v.xxx******xxx##.~.#xx#...
.vxx**********xxx..xxx.....
.v.xx***********xxxx.......
.v.xxx********...##........


## Think About...

This first assignment may not have been difficult for you if you've encountered A* search before in your Algorithms course. In preparation for future assignments that build on State Space Search, you can think about the following or even do an implementation if you like. You should **not** submit it as part of this assignment.

In several future assignments, we will have a need for a "plain ol'" Depth First Search algorithm.

1. Implement DFS Search to solve the problem presented in this programming assignment. Try to be as general as possible (don't hard code anything you can pass as a formal parameter).
2. Can you implement DFS Search as a higher order function and supply your own `is_goal`, `successors`, and `path` functions? How do you handle *state*?
3. Can you write a version of DFS that returns all the solutions?

In one future assignment a Breadth First Search algorithm will be very handy. Can you implement a search algorithm that changes whether it uses DFS or BFS by parameterization?

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.